[kloppyのサイト](https://kloppy.pysport.org/open-data/)みているときに、みつけたSkillCornerというところのトラッキングデータの中身をみてみる

In [1]:
from kloppy import skillcorner

ref: https://kloppy.pysport.org/getting-started/skillcorner/

In [ ]:
dataset = skillcorner.load(
    meta_data="../kloppy/kloppy/tests/files/skillcorner_match_data.json",
    raw_data="../kloppy/kloppy/tests/files/skillcorner_structured_data.json",
    
    # Optional arguments
#     sample_rate=1/10, # 1sec
#     limit=100,
    coordinates="skillcorner",
    include_empty_frames=False
)

In [ ]:
type(dataset)

In [ ]:
df_tracking = dataset.to_df()

In [ ]:
df_tracking.shape

In [ ]:
print(34783/90/60)

In [ ]:
df_tracking.columns

In [ ]:
df_tracking.head(5)

↑ なんで11.2秒から？？

In [ ]:
df_tracking.tail(5)

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', None)

In [ ]:
# df_tracking['timestamp'].head(1000)
df_tracking['timestamp'][120:130]

↑ 一気に飛ぶときがあるな、、、ボールがアウトしているときとかはもしかして省かれている？？

In [ ]:
df_tracking[120:130]

In [ ]:
# カラム調べる
print("\n".join(df_tracking.columns.tolist()))

In [ ]:
home_list = []
for data in df_tracking.columns.tolist():
    if 'home_' in data and '_anon_' not in data:
        home_list.append(data)
print(len(home_list) / 4)
print(home_list)

In [ ]:
away_list = []
for data in df_tracking.columns.tolist():
    if 'away_' in data and '_anon_' not in data:
        away_list.append(data)
print(len(away_list) / 4)
print(away_list)

# 特定のフレームをplotしてみる

In [ ]:
import matplotsoccer
import matplotlib.pyplot as plt

In [ ]:
se_first_frame = df_tracking.loc[0, :][df_tracking.loc[0, :].notna()]
se_first_frame

↑ 
https://github.com/SkillCorner/opendata#limitation

> The data is limited to what is visible on the broadcast video. Not all the players are visible (thus included in the data) all the time. The broadcast show an average of 14 players out of 22 at each frame. During replays or close up views, the data is not included.

StatsBombの360°データと同じ感じかな？？

In [ ]:
# import re

# matplotsoccer.field("green",figsize=8, show=False)

# pattern = pattern = '(.+_\d+)_.+'

# pos_dic = {} # key = away_7, value = (28.816779, -0.094319) としたい
# for i, v in se_first_frame.items():
#     if '_x' in i:
#         matched = re.match(pattern, i) # 例: home_4 をとる
#         y = se_first_frame[matched.group(1) + "_y"]
#         plt.scatter(v, y)
        
# plt.show()

matplotsoccerと特に整合性ないぽい！

https://kloppy.pysport.org/examples/plotting/
みて、別の探す（あるぽい）

In [ ]:
# !pip3 install mplsoccer

In [ ]:
from mplsoccer.pitch import Pitch

# ref: https://mplsoccer.readthedocs.io/en/latest/gallery/pitch_setup/plot_compare_pitches.html
pitch = Pitch(pitch_color='#e7f1fa', line_zorder=1, line_color='black', pitch_length=105, pitch_width=68, pitch_type="skillcorner")
fig, ax = pitch.draw()

sc = pitch.scatter([0, 20], [0, -10], ax=ax)

In [ ]:
import re

# pitch = Pitch(pitch_color='#e7f1fa', line_zorder=1, line_color='black', pitch_length=105, pitch_width=68, pitch_type="skillcorner")
# fig, ax = pitch.draw()

# pattern = pattern = '(.+_\d+)_.+'

# pos_dic = {} # key = away_7, value = (28.816779, -0.094319) としたい
# for i, v in se_first_frame.items():
#     if '_x' in i:
#         matched = re.match(pattern, i) # 例: home_4 をとる
#         y = se_first_frame[matched.group(1) + "_y"]
#         plt.scatter(v, y)
#         color = "#0000FF" if 'home_' in i else "#00FF00"
#         sc = pitch.scatter(v, y, ax=ax, c=color)

このフレームはボールがないからボールも描画できるようにする

In [ ]:
ball_exists_sr = df_tracking.loc[10, :][df_tracking.loc[10, :].notna()]
ball_exists_sr

In [ ]:
def drawFrame(sr):
    pitch = Pitch(pitch_color='#e7f1fa', line_zorder=1, line_color='black', pitch_length=105, pitch_width=68, pitch_type="skillcorner")
    fig, ax = pitch.draw(figsize=(6, 3))

    pattern = '(.+)_.+' #'(.+_\d+)_.+'

    pos_dic = {} # key = away_7, value = (28.816779, -0.094319) としたい
    for i, v in sr.items():
        if '_x' in i:
            matched = re.match(pattern, i) # 例: home_4 をとる
            y = sr[matched.group(1) + "_y"]
            color = "#FF0000" 
            if 'home_' in i:
                color = "#00FF00" 
            elif 'away_' in i:
                color = "#0000FF" 
            sc = pitch.scatter(v, y, ax=ax, c=color)

In [ ]:
drawFrame(ball_exists_sr)

In [ ]:
# for i in range(20):
#     sr = df_tracking.loc[i, :][df_tracking.loc[i, :].notna()]
#     drawFrame(sr)

matplotlibのアニメーションに挑戦！！！

In [ ]:
import matplotlib.animation as animation

In [ ]:
%matplotlib notebook

In [ ]:
# fig, ax = plt.subplots()

# def hoge(frame):
#     ax.clear()
    
#     # 毎回pitchを描画するしかない？？
#     pitch = Pitch(pitch_color='#e7f1fa', line_zorder=1, line_color='black', pitch_length=105, pitch_width=68, pitch_type="skillcorner")
#     pitch.draw(ax=ax)
    
#     sr = df_tracking.loc[frame, :][df_tracking.loc[frame, :].notna()]
    
#     ###
#     pattern = '(.+)_.+' #'(.+_\d+)_.+'

#     pos_dic = {} # key = away_7, value = (28.816779, -0.094319) としたい
#     for i, v in sr.items():
#         if '_x' in i:
#             matched = re.match(pattern, i) # 例: home_4 をとる
#             y = sr[matched.group(1) + "_y"]
#             color = "#FF0000" 
#             if 'home_' in i:
#                 color = "#00FF00" 
#             elif 'away_' in i:
#                 color = "#0000FF" 
#             sc = pitch.scatter(v, y, ax=ax, c=color)
#     ###
    
# ani = animation.FuncAnimation(fig, hoge, frames=range(60), interval=100, repeat=False)

# plt.show()

アニメーションしばらく流している、streamがcloseした、みたいなエラーになる。
感覚的にはメモリが足りてない？

メモリめっちゃ使ってたDockerを終了させたらより動いた

In [ ]:
fig, ax = plt.subplots()

list=[] # [(x,y,c)] みたいなものを入れる配列

frames = 100

for i in range(frames):
    sub_list = []
    sr = df_tracking.loc[i, :][df_tracking.loc[i, :].notna()]
    for i, v in sr.items():
        if '_x' in i:
            pattern = '(.+)_.+'
            matched = re.match(pattern, i)
            y = sr[matched.group(1) + "_y"]
            color = "#FF0000" 
            if 'home_' in i:
                color = "#00FF00" 
            elif 'away_' in i:
                color = "#0000FF" 
            sub_list.append((v,y,color))
    list.append(sub_list)
    
def hoge(frame):
    ax.clear()
    
    # 毎回pitchを描画するしかない？？
    pitch = Pitch(pitch_color='#e7f1fa', line_zorder=1, line_color='black', pitch_length=105, pitch_width=68, pitch_type="skillcorner")
    pitch.draw(ax=ax)

    for sub in list[frame]:
            sc = pitch.scatter(sub[0], sub[1], ax=ax, c=sub[2])
    
ani = animation.FuncAnimation(fig, hoge, frames=range(frames), interval=100, repeat=False)

plt.show()